In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Input and output directories
input_directory = "formatted_data"
output_directory = "cleaned_data"

# Create a folder for cleaned data
os.makedirs(output_directory, exist_ok=True)

# Function to detect and remove outliers using IQR
def remove_outliers(df, x_col, y_col):
    # Calculate Interquartile Range (IQR) for x and y columns
    Q1_x = df[x_col].quantile(0.25)
    Q3_x = df[x_col].quantile(0.75)
    IQR_x = Q3_x - Q1_x

    Q1_y = df[y_col].quantile(0.25)
    Q3_y = df[y_col].quantile(0.75)
    IQR_y = Q3_y - Q1_y

    # Define bounds for x and y
    lower_bound_x = Q1_x - 1.5 * IQR_x
    upper_bound_x = Q3_x + 1.5 * IQR_x

    lower_bound_y = Q1_y - 1.5 * IQR_y
    upper_bound_y = Q3_y + 1.5 * IQR_y

    # Filter data within the bounds
    filtered_df = df[(df[x_col] >= lower_bound_x) & (df[x_col] <= upper_bound_x) &
                     (df[y_col] >= lower_bound_y) & (df[y_col] <= upper_bound_y)]
    
    # Detect and remove isolated points and false peaks
    smoothed_y = filtered_df[y_col].rolling(window=5, center=True).mean()
    residuals = np.abs(filtered_df[y_col] - smoothed_y)
    residual_threshold = 1.5 * residuals.std()
    filtered_df = filtered_df[residuals <= residual_threshold]
    
    print(f"Data before filtering: {df.shape[0]} rows")
    print(f"Data after filtering: {filtered_df.shape[0]} rows")
    return filtered_df

# Process each file in the folder
for file_name in os.listdir(input_directory):
    if file_name.endswith(".csv"):
        file_path = os.path.join(input_directory, file_name)

        # Read the CSV file with the correct delimiter
        df = pd.read_csv(file_path, delimiter=';')

        # Ensure the relevant columns are numeric
        df['Acceleration voltage U_B / V'] = pd.to_numeric(df['Acceleration voltage U_B / V'], errors='coerce')
        df['Collector current I_A / nA'] = pd.to_numeric(df['Collector current I_A / nA'], errors='coerce')

        # Drop rows with NaN values in the relevant columns
        df = df.dropna(subset=['Acceleration voltage U_B / V', 'Collector current I_A / nA'])

        # Remove outliers
        cleaned_df = remove_outliers(df, 'Acceleration voltage U_B / V', 'Collector current I_A / nA')
        
        # Save the cleaned data
        cleaned_file_path = os.path.join(output_directory, file_name)
        cleaned_df.to_csv(cleaned_file_path, index=False)
        
        # Read all cleaned CSV data for verification
        df = pd.read_csv(cleaned_file_path)
        print(f"Processed file: {file_name}")
        print(df)

print("Data cleaning completed. Cleaned files is saved in the 'cleaned_data' folder.")

Data before filtering: 401 rows
Data after filtering: 369 rows
Processed file: 1.5 6.0 2.csv
     Time t / s  Voltage U_A1 / V  Voltage U_B1 / V  \
0         0.300             0.480             0.100   
1         0.401             0.495             0.130   
2         0.500             0.495             0.155   
3         0.599             0.495             0.185   
4         0.699             0.510             0.215   
..          ...               ...               ...   
364      39.399            12.855             7.910   
365      39.499            12.855             7.910   
366      39.599            12.855             7.910   
367      39.699            12.855             7.910   
368      39.801            12.855             7.910   

     Collector current I_A / nA  Acceleration voltage U_B / V  
0                         0.480                          1.00  
1                         0.495                          1.30  
2                         0.495                       